In [2]:
!rm -rf .models
!mkdir .models
!wget -q -O .models/cifar_nat.pt "https://www.dropbox.com/s/yhpp4yws7sgi6lj/cifar_nat.pt?dl=1"

In [1]:
import os
import pathlib
import random
import functools
import copy
import sys
import pickle
import tarfile
import operator
import math
import requests
import importlib
os.environ["NOTEBOOK_MODE"] = "1"

import tqdm
import cox

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import sklearn
from sklearn import preprocessing
import scipy

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

import flax
import neural_tangents as nt
import art
import robustness
from robustness import model_utils, datasets, train, defaults

import vit_jax

import adversarial_ntks as atk

In [2]:
CIFAR10 = {
    "dataset": robustness.datasets.CIFAR(".data"),
    "workers": 12,
    "arch": "resnet50",
    "resume_path": ".models/cifar_nat.pt",
    "labels": ["airplane", "automobile", "bird", "cat", "deer",
        "dog", "frog", "horse", "ship", "truck"],
    
    "train_subset": None,
    "test_subset": 512,
    "batch_size": 32,
    
    "perturb_target": 3,
    "perturb_eps": 0.02,

    "display_samples": 10,
    "display_sample_img_size": 3}

In [3]:
def exp_make_loaders(exp):
    exp["train_loader"], exp["val_loader"] = exp["dataset"].make_loaders(
        batch_size=exp["batch_size"],
        test_subset=exp["test_subset"],
        workers=exp["workers"])

exp_make_loaders(CIFAR10)

==> Preparing dataset cifar..
Files already downloaded and verified
Files already downloaded and verified


In [4]:
def exp_load_saved(exp):
    exp["model"], _ = model_utils.make_and_restore_model(
        arch=exp["arch"],
        resume_path=exp["resume_path"],
        dataset=exp["dataset"])
    exp["model"].eval()

exp_load_saved(CIFAR10)

=> loading checkpoint '.models/cifar_nat.pt'
=> loaded checkpoint '.models/cifar_nat.pt' (epoch 190)


In [5]:
def exp_eval(exp):
    exp.update({
        "eval_args": defaults.check_and_fill_args(
            args=cox.utils.Parameters({
                "adv_eval": 1,
                "out_dir": ".results",
                "constraint": "inf",
                "eps": exp["perturb_eps"],
                "attack_lr": 2.5 * exp["perturb_eps"] / 20,
                "attack_steps": 20}),
            arg_list=defaults.PGD_ARGS,
            ds_class=datasets.CIFAR),
        "cox": cox.store.Store(".cox")})
    exp["model"].eval()
    exp["eval_results"] = train.eval_model(
        model=exp["model"],
        args=exp["eval_args"],
        loader=exp["val_loader"],
        store=exp["cox"])
    print(exp["eval_results"])

exp_eval(CIFAR10)

Logging in: c:\Users\0\main\active\erlija\adversarial-ntks\notebooks\uniform-e-balls\.cox\f2c46291-b1c7-4103-8552-720ed5e9db56


  0%|          | 0/313 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "c:\Users\0\main\active\erlija\adversarial-ntks\.venv\lib\site-packages\IPython\core\interactiveshell.py", line 3427, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-0aea40674bf3>", line 22, in <module>
    exp_eval(CIFAR10)
  File "<ipython-input-5-0aea40674bf3>", line 15, in exp_eval
    exp["eval_results"] = train.eval_model(
  File "f:\users\0\main\active\erlija\adversarial-ntks\robustness\robustness\train.py", line 160, in eval_model
    adv_prec1, adv_loss = _model_loop(args, 'val', loader,
  File "f:\users\0\main\active\erlija\adversarial-ntks\robustness\robustness\train.py", line 447, in _model_loop
    output, final_inp = model(inp, target=target, make_adv=adv,
  File "c:\Users\0\main\active\erlija\adversarial-ntks\.venv\lib\site-packages\torch\nn\modules\module.py", line 727, in _call_imp

TypeError: object of type 'NoneType' has no len()

In [ ]:
def exp_eval_finetune(exp):
    exp.update({
        "finetune_args": defaults.check_and_fill_args(
            args=cox.utils.Parameters({
                "adv_train": 1,
                "out_dir": ".results",
                "constraint": "inf",
                "eps": exp["perturb_eps"],
                "attack_lr": 2.5 * exp["perturb_eps"] / 20,
                "attack_steps": 20}),
            arg_list=defaults.PGD_ARGS,
            ds_class=datasets.CIFAR),
        "finetune_params": exp["model"].model.parameters()})
    exp["finetune_args"] = defaults.check_and_fill_args(
        defaults.check_and_fill_args(
            exp["finetune_args"],
            defaults.TRAINING_ARGS,
            datasets.CIFAR),
        defaults.PGD_ARGS,
        datasets.CIFAR)
    exp["model"].train()
    exp["finetune_results"] = train.eval_model(
        model=exp["model"],
        update_params=exp["finetune_params"],
        args=exp["finetune_args"],
        loadesr=(exp["train_loader"], exp["val_loader"]),
        store=exp["cox"]))
    print(exp["finetune_results"])

exp_eval_finetune(CIFAR10)